# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []


# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


619

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
#base url
base_url = "https://api.openweathermap.org/data/2.5/weather?"

#create query url
query_url = f"{base_url}&appid={weather_api_key}&q="

#create lists
lng = []
lat = []
temp = []
cloudiness = []
humidity = []
wind_speed = []
city_id = []
city_name = []
country = []

#loop through cities
for city in cities:
    
    #create json response
    response = requests.get(query_url + city).json()
    
    
    #try/except
    try:
    
        lng.append(response['coord']['lon'])
        lat.append(response['coord']['lat'])
        temp.append(response['main']['temp'])
        cloudiness.append(response['clouds']['all'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        city_id.append(response['id'])
        city_name.append(response['name'])
        country.append(response['sys']['country'])
        
        print(f"City Name: {city}")
        
    except:
        
        print("city does not exist")


City Name: tasiilaq
City Name: luba
City Name: porbandar
City Name: rikitea
city does not exist
City Name: qaanaaq
City Name: pasighat
City Name: robe
city does not exist
City Name: castro
City Name: dalbandin
City Name: georgetown
City Name: sao filipe
City Name: saint george
City Name: ponta do sol
City Name: hobart
City Name: ushuaia
City Name: elat
City Name: okha
city does not exist
City Name: ilulissat
City Name: pouembout
City Name: chokurdakh
City Name: dikson
City Name: petropavlovsk-kamchatskiy
City Name: cockburn town
City Name: mataura
City Name: chuy
City Name: albany
city does not exist
City Name: poum
City Name: nanortalik
City Name: lakes entrance
City Name: yellowknife
City Name: kodiak
City Name: nioro
city does not exist
City Name: muzhi
City Name: christchurch
City Name: puerto penasco
City Name: jalu
City Name: saint-augustin
City Name: boma
City Name: vallenar
City Name: berlevag
city does not exist
City Name: carnarvon
City Name: cabinda
City Name: lompoc
City Na

City Name: quesnel
City Name: lieksa
City Name: sayyan
City Name: miragoane
City Name: port augusta
City Name: kuito
City Name: gavle
City Name: sola
City Name: yar-sale
City Name: xinzhi
City Name: mabaruma
City Name: camacha
City Name: komyshnya
City Name: luwuk
City Name: tilichiki
City Name: voljevac
City Name: sambava
City Name: nhulunbuy
City Name: copiapo
City Name: nova olinda do norte
City Name: diffa
City Name: noumea
City Name: viedma
City Name: sitka
City Name: alyangula
city does not exist
City Name: stromness
city does not exist
City Name: souillac
City Name: himatangi
City Name: garbolovo
City Name: sioux lookout
City Name: kiama
City Name: mangan
City Name: deogarh
City Name: yuksekova
City Name: port moresby
City Name: tabriz
City Name: salinopolis
City Name: luderitz
City Name: sao jose dos pinhais
City Name: padang
City Name: ayan
city does not exist
City Name: leh
City Name: santa cruz cabralia
City Name: svetlogorsk
City Name: arman
City Name: parambu
City Name: sa

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [10]:
weatherpy_dict = {"City ID": city_id,
                  "City Name": city_name,
                  "Longitude":lng,
                  "Latitude":lat,
                  "Cloudiness":cloudiness,
                  "Temp":temp,
                  "Humidity":humidity,
                  "Wind Speed":wind_speed,
                  "Country":country
}


weatherpy_df = pd.DataFrame(weatherpy_dict)
weatherpy_df.head()

,City ID,City Name,Longitude,Latitude,Cloudiness,Temp,Humidity,Wind Speed,Country
0,3424607,Tasiilaq,-37.6368,65.6145,3,279.24,64,1.62,GL
1,2309528,Luba,8.5547,3.4568,100,297.09,84,3.30,GQ
2,1259395,Porbandar,69.6093,21.6422,75,301.14,83,6.17,IN
3,4030556,Rikitea,-134.9692,-23.1203,19,296.16,73,11.22,PF
4,3831208,Qaanaaq,-69.3632,77.4840,33,279.06,66,2.63,GL


In [11]:
# Export to .csv file
weatherpy_df.to_csv("weatherpy.csv")

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
#  Get the indices of cities that have humidity over 100%.


In [7]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression